## Install Requirements

An example of using Chroma DB and LangChain to do question answering over documents, with a locally persisted database. You can store embeddings and documents, then use them again later.

In [10]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import CohereEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.llms import Cohere
from langchain.chains import VectorDBQA
from langchain.document_loaders import TextLoader

## Load and process documents
Load documents to do question answering over. If you want to do this over your documents, this is the section you should replace.

Next we split documents into small chunks. This is so we can find the most relevant chunks for a query and pass only those into the LLM.

In [11]:
# Load and process the text
loader = TextLoader('state_of_the_union.txt')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [12]:
COHERE_API_KEY="key"

## Initialize PeristedChromaDB
Create embeddings for each chunk and insert into the Chroma vector database. The persist_directory argument tells ChromaDB where to store the database when it's persisted.

In [13]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

embedding = CohereEmbeddings(cohere_api_key=COHERE_API_KEY)
vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)

Using embedded DuckDB with persistence: data will be stored in: db


## Persist the Database
In a notebook, we should call persist() to ensure the embeddings are written to disk. This isn't necessary in a script - the database will be automatically persisted when the client object is destroyed.

In [14]:
vectordb.persist()
vectordb = None

## Load the Database from disk, and create the chain
Be sure to pass the same persist_directory and embedding_function as you did when you instantiated the database. Initialize the chain we will use for question answering.

In [15]:
# Now we can load the persisted database from disk, and use it as normal. 
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
qa = VectorDBQA.from_chain_type(llm=Cohere(cohere_api_key=COHERE_API_KEY), chain_type="stuff", vectorstore=vectordb)

Using embedded DuckDB with persistence: data will be stored in: db
/Users/sager/anaconda3/lib/python3.10/site-packages/langchain/chains/retrieval_qa/base.py:201: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


## Ask questions!
Now we can use the chain to ask questions!

In [16]:
query = "What did the president say about Ketanji Brown Jackson"
qa.run(query)

" The president said that Ketanji Brown Jackson was one of the nation's top legal minds who would continue Justice Breyer's legacy of excellence."

## Cleanup
When you're done with the database, you can delete it from disk. You can delete the specific collection you're working with (if you have several), or delete the entire database by nuking the persistence directory.

In [17]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# Or just nuke the persist directory
!rm -rf db/